In [1]:
# 참조: https://medium.com/@yharsh800/boston-housing-cross-validation-35058593613b
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

# boston = datasets.load_boston()
# boston.data.shape, boston.target.shape
test = pd.read_csv("./test.csv")
train = pd.read_csv("./train.csv")

In [2]:
y = train['price']
del train['price']

In [3]:
train_len = len(train) # 둘을 합치기 전에, 구분점(train의 길이)을 저장
data = pd.concat((train, test), axis=0)  # 합침

In [4]:
sub_id = data['id'][train_len:] # train만 저장
del data['id']

In [5]:
data['date'] = data['date'].apply(lambda x : str(x[:6])).astype(float) # 앞에 6개만 자르고, str로 바꿔서 저장
data.tail()

,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
6463,201406.0,3,1.75,1500,11968,1.0,0,0,3,6,1500,0,2014,0,98010,47.3095,-122.002,1320,11303
6464,201501.0,3,2.00,1490,1126,3.0,0,0,3,8,1490,0,2014,0,98144,47.5699,-122.288,1400,1230
6465,201502.0,3,2.50,1310,1294,2.0,0,0,3,8,1180,130,2008,0,98116,47.5773,-122.409,1330,1265
6466,201406.0,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
6467,201501.0,3,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [6]:
sub = data.iloc[train_len:, :] # test
x = data.iloc[:train_len, : ] # train

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.4, random_state=10)

print(X_train.shape, y_train.shape)

print(X_test.shape, y_test.shape)

(9021, 19) (9021,)
(6014, 19) (6014,)


In [9]:
clf = svm.SVC(kernel='rbf', C=1, gamma=0.1) 
clf.fit(X_train, y_train) 
# regression = svm.SVR(kernel='linear').fit(X_train, y_train)
clf.score(X_test, y_test)

0.0068174260059860325

In [10]:
Y_prediction = clf.predict(sub).flatten()

In [11]:
sub = pd.DataFrame(data={'id':sub_id, 'price':Y_prediction})

In [12]:
sub.to_csv("1st_rbf_SVM.csv", index=False)

In [ ]:
from sklearn.model_selection import cross_val_score
regression = svm.SVR(kernel='linear', C=1)
scores = cross_val_score(regression, boston.data, boston.target, cv=5)
scores

In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()*2))

In [ ]:
from sklearn import metrics
scores = cross_val_score(
    regression, boston.data, boston.target, cv=5,
scoring='neg_mean_squared_error')
scores

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

pipe_svm = make_pipeline(StandardScaler(),
                        PCA(n_components=2),
                        svm.SVR(kernel='linear', C=1))
pipe_svm.fit(X_train, y_train)


In [ ]:
y_pred = pipe_svm.predict(X_test)
print('Test Accuracy: %.3f' % pipe_svm.score(X_test, y_test))

In [ ]:
scores = cross_val_score(estimator=pipe_svm,
                        X=X_train,
                        y=y_train,
                        cv=10,
                        n_jobs=1)
print('CV accuracy scores: %s' % scores)

In [ ]:
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),
                                     np.std(scores)))

1st rbf_svm  - 382,692 - 04/06